# 1、Agent理解

## 1.1、什么是Agent

- 在大模型应用中, Agent是具备"自主决策能力"【可动态协调大模型和工具来完成复杂任务的智能系统】的智能体,能够根据用户需求,自主分析问题、选择工具、执行操作【让LLM充当决策大脑,根据用户输入自主选择和执行工具(如:搜索、计算、查询数据库等)】,直到完成任务,生成精准响应
- 举例说明,当问"明天深圳适合穿短袖吗",普通大模型可能会因为数据时效性无法回答,而Agent操作如下:
    - 1）分析问题: 明确需要获取"明天深圳的天气"和"适合穿短袖的天气"两个信息
    - 2）选择工具: 调用 查询天气API并获取气温数据
    - 3）执行计算: 对比气温和适合穿短袖的范围
    - 4）生成回答: 整合结果给出是否 适合穿短袖的建议
- 核心区别: 普通大模型会直接回答问题(可能会随意编造,存在"幻觉"问题);Agent会基于"自主决策+工具调用"来主动解决实际问题

## 1.2、Agent的核心组件/能力

- 1）大模型(LLM): 作为大脑,提供推理、规划和知识理解能力【如: OpenAI()、ChatOpenAI()】
- 2）工具(Tools): 具备短期记忆(上下文) 和 长期记忆【向量存储】, 支持快速检索知识
    - 如: ConversationBufferMemory()、ConversationSummaryMemory()、ConversationBufferWindowMemory()
    - 如: 使用Faiss、Milvus等向量数据库来实现持久化
- 3）工具(Tools): 调用外部工具(如 API、数据库)的执行单元
- 4）规划(Planning): 逐步分解任务、反思和自省,以处理复杂任务
- 5）行动(Action): 实际执行决策的能力
- 6）执行器(Executor): 调度中枢,负责协调LLM 和 工具的交互流程,驱动"决策=》工具调用=》结果反馈"的迭代循环
- 7）协作: 基于A2A协议,与其他智能体交互合作,完成更复杂的任务目标

- 问题: 为什么要调用第三方工具(比如: 搜索引擎或数据库) 或 借助第三方库?
- 回答: 大模型虽然强大,但也具备一定的局限性(不能回答时效性高的问题、处理复杂的数学逻辑等问题),可借助第三方工具来扩展大模型的能力

## 1.3、Agent类型

- 在LangChain中Agent类型就是针对不同的业务场景,为开发者提供的解决方案。API说明：https://python.langchain.com/v0.1/docs/modules/agents/agent_types/
- Agents的核心类型有两种模式:
    - 方式1: Funcation Call模式
        - 基于 结构化函数调用 （如 OpenAI Function Calling）
        - 直接生成工具调用参数（ JSON 格式 ）
        - 效率更高，适合工具明确的场景
    - 方式2: ReAct 模式
        - 基于 文本推理 的链式思考（Reasoning + Acting），具备反思和自我纠错能力
            - 推理（Reasoning）：分析当前状态，决定下一步行动
            - 行动（Acting）：调用工具并返回结果
        - 通过 自然语言描述决策过程 =》适合需要明确推理步骤的场景,如智能客服、问答系统、任务执行等

# 2、Agent代码实践

## 2.1、FUNCATION_CALL模式调用

- 核心思想: LLM通过输出结构化指令(如JSON),明确指定要调用的工具名称和参数,Agent执行器 解析该指令并调用工具, 最后将结果返回给LLM生成最终回答
    - "LLM生成工具调用指令,执行器负责执行",LLM不直接处理工具返回结果的格式,仅专注于"是否调用工具"和"如何调用"
    - 依赖LLM的结构化输出能力(如 OpenAI的Function Calling、Anthropic的Tool Use等),无需额外的提示词模板引导格式,鲁棒性强,不易出错
- 工作流程
    - 1）用户输入: 用户提出问题
    - 2）LLM决策: LLM分析问题后,生成包含工具名称和参数的结构化指令(如JSON)
    - 3）工具调用: 执行器解析指令,调用对应的工具,获取返回结果
    - 4）整理结果：LLM接收工具返回的结果,将其整理成自然语言后 回答给用户
- 适用场景
    - 不需要复杂的多轮工具调用逻辑(仅需单次工具调用即可解决问题)
    - 工具返回固定格式的结果(如API返回JSON、数据库查询返回表格)
    - 使用支持函数调用的LLM(如OpenAI、Anthropic)

In [2]:
from langchain_classic.agents import create_tool_calling_agent, AgentExecutor, create_react_agent
from langchain_core.tools import Tool
from langchain_openai import ChatOpenAI
import os, dotenv
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.prompts import ChatPromptTemplate

dotenv.load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")
os.environ['OPENAI_BASE_URL'] = os.getenv("OPENAI_BASE_URL")
llm = ChatOpenAI(model="gpt-4o-mini",temperature=0)

# 1.定义检索工具
# 1.1.设置搜索工具Tavily的API KEY
# 在 .env 中添加配置的TAVILY_API_KEY = "tvly-dev-xxx"
os.environ["TAVILY_API_KEY"] = os.getenv("TAVILY_API_KEY")

# 1.2.初始化Tavily实例,并获取前三个结果
tavily_search = TavilySearchResults(max_results=3)   # 默认原工具名为tavily_search_results_json

# 2. 创建Tool的实例(本步骤可以考虑省略，直接使用[tavily_search]替换[tavily_tool]
#    但加上可根据业务指定工具的名称、描述和调用函数
tavily_tool = Tool(
    name="tavily_search_tool",
    func=tavily_search.run,
    description="联网检索信息的工具"
)
tools = [tavily_tool]
# 3.自定义提示词模板
prompt = ChatPromptTemplate.from_messages([
    ("system","您是一位乐于助人的助手，必要时可使用 tavily_search_tool 工具来获取信息"),
    ("human","{input}"),
    ("system","{agent_scratchpad}"),  # 存储和传递Agent的思考过程
    # ("placeholder","{agent_scratchpad}")
    # 不加会报错,ValueError: Prompt missing required variables: {'agent_scratchpad'}
])

# 4.创建Agent对象
agent = create_tool_calling_agent(
    llm=llm,
    tools=tools,
    prompt=prompt,
)

# 5.创建AgentExecutor执行器(verbose=True,则会在控制台显示详细的推理过程)
agent_executor =AgentExecutor(agent=agent,tools=tools,verbose=True)

# 6.调用执行器完成任务,得到响应
agent_executor.invoke({"input":"请根据深圳未来三天的天气生成一份旅游攻略"})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_tool` with `深圳未来三天的天气预报`


[{'title': '天气实况与预报-深圳市气象局（台）', 'url': 'https://weather.sz.gov.cn/qixiangfuwu/yubaofuwu/index.html', 'content': '2025年11月06日 周四 23:57:55（农历九月十七）\n\n繁體 "以繁體中文瀏覽")\nEnglish\n手机版\n数据开放\n无障碍阅读 "无障碍阅读")\n进入关怀版)\n\n热门搜索：\n天文台预约\n气象材料证明\n气象数据下载\n\n### 预报服务\n\n天气实况与预报\n\n天气提示\n\n天气解读\n\n城市天气\n\n当前位置 :\n首页 > 气象服务 > 预报服务 > 天气实况与预报\n\n深圳国家基本气象站实况\n12:54\n\n石岩梯度塔\n\n当前预警\n更多\n\n天气分布图\n降雨估测\n\n26\n\n气温 ℃\n\n东风小于三级\n\n风\n\n30\n\n能见度 km\n\n1007.4\n\n气压 hPa\n\n0\n\n日雨量 mm\n\n69\n\n相对湿度 %\n\n【深圳天气趋势】预计8-9日多云间晴天，气温上升；10-13日云量增多，间中有零星小雨，气温逐日下降，早晚清凉。   深圳市气象台2025年11月07日10:00发布。\n\n## 深圳各区逐时预报\n\n###### 11月7日 10:05 更新\n\n罗湖\n福田\n南山\n宝安\n龙岗\n龙华\n光明\n坪山\n盐田\n大鹏\n深汕\n\n福田区\n\n# 23-28℃\n\n阴天间多云，偶有零星小雨\n\n 60-80 [...] | 预报日期 | 今天（11月6日） | 明天（11月7日） | 后天（11月8日） |\n ---  --- |\n| 空气污染气象条件等级预报 | 二级 较好较有利于空气污染物稀释、扩散和清除 | 三级 一般对空气污染物稀释、扩散和清除无明显影响 | 二级 较好较有利于空气污染物稀释、扩散和清除 |\n| 灰霾等级预报 | 无 灰霾能见度良好 | 无 灰霾能见度良好 | 无 灰霾能见度良好 |\n\n#

{'input': '请根据深圳未来三天的天气生成一份旅游攻略',
 'output': '根据深圳未来三天的天气预报，以下是为您准备的旅游攻略：\n\n### 深圳未来三天天气概况\n- **11月7日（周四）**: 多云，气温在23-28℃之间，偶有零星小雨，适合户外活动。\n- **11月8日（周五）**: 多云，气温在22-27℃，风速较小，适合游玩。\n- **11月9日（周六）**: 多云，气温在21-26℃，天气较为舒适。\n\n### 旅游攻略\n\n#### 第一天（11月7日）\n- **上午**: \n  - **深圳湾公园**: 享受晨间的清新空气，适合晨跑或散步。\n- **中午**: \n  - **南山海上世界**: 在这里享用海鲜午餐，欣赏海景。\n- **下午**: \n  - **深圳博物馆**: 了解深圳的历史与文化，适合雨天的室内活动。\n- **晚上**: \n  - **华侨城创意文化园**: 逛逛艺术展览，享受夜晚的氛围。\n\n#### 第二天（11月8日）\n- **上午**: \n  - **大梅沙海滨公园**: 享受阳光和海滩，适合游泳和沙滩活动。\n- **中午**: \n  - **大梅沙附近的海鲜餐厅**: 尝试当地新鲜的海鲜。\n- **下午**: \n  - **深圳东部华侨城**: 游玩主题公园，适合家庭游客。\n- **晚上**: \n  - **深圳欢乐谷**: 体验夜间游乐设施，享受刺激的游乐项目。\n\n#### 第三天（11月9日）\n- **上午**: \n  - **莲花山公园**: 爬山健身，俯瞰深圳市区的美景。\n- **中午**: \n  - **福田区的美食街**: 尝试各种地方小吃。\n- **下午**: \n  - **深圳图书馆**: 享受安静的阅读时光，适合文化爱好者。\n- **晚上**: \n  - **深圳中心公园**: 散步放松，享受夜晚的城市灯光。\n\n### 注意事项\n- **穿衣建议**: 建议穿着轻便的夏季服装，早晚温差较大，可以准备一件薄外套。\n- **防雨准备**: 虽然天气以多云为主，但偶尔会有小雨，建议携带雨具。\n- **防晒措施**: 紫外线指数中等，外出时请注意防晒。\n\n希望这份攻略能帮助您在深圳度过一个愉快的假期！'}

- 额外说明:
    - 在声明提示词模板时,agent_scratchpad变量必须要有,agent_scratchpad用于存储和传递Agent的思考过程,如: 在调用链式工具时(如先搜索天气再推荐行程),agent_scratchpad会保留所有历史步骤,避免丢失上下文

## 2.2、ReAct模式调用

- 核心思想: LLM通过自然语言推理("思考过程") 和 工具调用指令(结构化格式)的结合,逐步拆解问题、调用工具,并根据工具返回的结果迭代优化, 生成最终的回答
    - "LLM 模拟人类思考过程,边想边做",将"思考"和"工具调用"显示地体现在输出中,执行器根据"工具调用指令"执行工具,并将结果反馈给LLM继续思考
    - 不依赖于LLM的原生函数调用能力,而是通过提示词模板引导LLM输出固定格式的"思考过程+工具调用"
- 工作流程
    - 1）用户输入: 用户提出问题（如 "谁是 2023 年诺贝尔物理学奖得主？他们的主要贡献是什么?")
    - 2）LLM思考和决策: LLM输出思考过程(如: "我需要先确认2023年诺贝尔物理学奖的得主"),并生成工具调用指令(如: "搜索: 2023年诺贝尔物理学奖得主")
    - 3）工具调用: 执行器解析工具调用指令,调用搜索工具,获取其返回结果(如"得主是...,其主要贡献是...")
    - 4）LLM迭代思考: LLM接收工具返回结果,继续输出思考过程(如"现在需要补充他们的主要贡献细节"),并生成第二次工具调用指令
    - 5）重复循环: 直到LLM认为已获取到足够的内容后,输出最终回答
- 优缺点:
    - 优点:
        - 灵活性强: LLM可根据工具返回结果动态调整思考方向,支持复杂的多轮工具调
        - 可解释性高: 显示输出思考过程,便于调试和理解Agent的决策逻辑
        - 适配范围广: 不依赖LLM的原生函数调用能力,通过提示词模板 即可适配各类LLM
    - 缺点:
        - 提示词复杂: 需要设计详细、精确的提示词模板(如: 引导LLM输出"Thought: ...\n Action: ...\n Action Input: ..."格式),否则容易出现解析错误
        - 对LLM能力要求高: 需要LLM具备较强的推理 和 格式遵循能力,否则可能会生成无效的工具调用指令
- 适用场景
    - 问题需要复杂的多轮工具调用(如: "先查数据/联网搜索,再计算分析,最后总结")
    - 需要展示Agent详细的思考过程
    - 使用不支持函数调用的LLM(如: Llama2/3)

### 1）使用PromptTemplate

- 在提示词中 要有可以使用的工具【tools、tool_names】、用户输入【input】和agent_scratchpad
- 远程的提示词模板要通过`https://smith.langchain.com/hub/hwchase17` 获取, 如：https://smith.langchain.com/hub/hwchase17/react 模板是专为ReAct模式设计的提示模板,该模板中已有聊天对话键tools、tools_name、input、agent_scratchpad,也可以使用LangChain Hub中的官方ReAct提示模板

- 复制下前面`Function Call`模式的代码，关键修改如下：
    - 添加含有tools、tools_name、input、agent_scratchpad的提示词模板
    - 使用create_react_agent()函数创建ReAct模式的Agent

In [12]:
from langchain_classic import hub
from langchain_classic.agents import AgentExecutor, create_react_agent
from langchain_core.tools import Tool
from langchain_openai import ChatOpenAI
import os, dotenv
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate

dotenv.load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")
os.environ['OPENAI_BASE_URL'] = os.getenv("OPENAI_BASE_URL")
llm = ChatOpenAI(model="gpt-4o-mini",temperature=0)

# 1.定义检索工具
# 1.1.设置搜索工具Tavily的API KEY
os.environ["TAVILY_API_KEY"] = os.getenv("TAVILY_API_KEY")

# 1.2.初始化Tavily实例,并获取前三个结果
tavily_search = TavilySearchResults(max_results=3)   # 默认原工具名为tavily_search_results_json

# 2. 创建Tool的实例(本步骤可以考虑省略，直接使用[tavily_search]替换[tavily_tool]
#    但加上可根据业务指定工具的名称、描述和调用函数
tavily_tool = Tool(
    name="tavily_search_tool",
    func=tavily_search.run,
    description="联网检索信息的工具"
)
tools = [tavily_tool]

# 3.自定义提示词模板(用于Function Call模式)
# prompt = ChatPromptTemplate.from_messages([
#     ("system","您是一位乐于助人的助手，必要时可使用 tavily_search_tool 工具来获取信息"),
#     ("human","{input}"),
#     ("placeholder","{agent_scratchpad}")  # 存储和传递Agent的思考过程
#     # 不加会报错,ValueError: Prompt missing required variables: {'agent_scratchpad'}
# ])

# 3.自定义提示词模版(用于ReAct模式,要有tools、tool_names、input、agent_scratchpad)
  # 通过 https://smith.langchain.com/hub/hwchase17/react获取提示词模板
prompt_template = '''Answer the following questions as best you can. You have access to the following
tools:
    {tools}
    Use the following format:
    Question: the input question you must answer
    Thought: you should always think about what to do
    Action: the action to take, should be one of [{tool_names}]
    Action Input: the input to the action
    Observation: the result of the action
    ... (this Thought/Action/Action Input/Observation can repeat N times)
    Thought: I now know the final answer
    Final Answer: the final answer to the original input question
    Begin!
    Question: {input}
    Thought:{agent_scratchpad}'''

prompt = PromptTemplate.from_template(prompt_template)

# 3.使用LangChain Hub中的官方ReAct提示模板
# prompt = hub.pull("hwchase17/react")

# 4.创建ReAct模式(反思和自我纠错能力)的Agent对象
# agent = create_tool_calling_agent(
agent = create_react_agent(
    llm=llm,
    tools=tools,
    prompt=prompt,
)

# 5.创建AgentExecutor执行器(verbose=True,则会在控制台显示详细的推理过程)
# 关键修改: 加入 handle_parsing_errors=True 参数，避免解析错误直接中断
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True,
                               handle_parsing_errors=True,
                               max_iterations=6 # 可选：限制最大迭代次数，防止无限循环
)

# 6.调用执行器完成任务,得到响应
agent_executor.invoke({"input": "请根据深圳未来三天的天气生成一份旅游攻略"})



> Entering new AgentExecutor chain...
我需要获取深圳未来三天的天气信息，以便为旅游攻略提供准确的建议。  
Action: tavily_search_tool  
Action Input: "深圳未来三天的天气预报"  [{'title': '天气实况与预报-深圳市气象局（台）', 'url': 'https://weather.sz.gov.cn/qixiangfuwu/yubaofuwu/index.html', 'content': '2025年11月06日 周四 23:57:55（农历九月十七）\n\n繁體 "以繁體中文瀏覽")\nEnglish\n手机版\n数据开放\n无障碍阅读 "无障碍阅读")\n进入关怀版)\n\n热门搜索：\n天文台预约\n气象材料证明\n气象数据下载\n\n### 预报服务\n\n天气实况与预报\n\n天气提示\n\n天气解读\n\n城市天气\n\n当前位置 :\n首页 > 气象服务 > 预报服务 > 天气实况与预报\n\n深圳国家基本气象站实况\n12:54\n\n石岩梯度塔\n\n当前预警\n更多\n\n天气分布图\n降雨估测\n\n26\n\n气温 ℃\n\n东风小于三级\n\n风\n\n30\n\n能见度 km\n\n1007.4\n\n气压 hPa\n\n0\n\n日雨量 mm\n\n69\n\n相对湿度 %\n\n【深圳天气趋势】预计8-9日多云间晴天，气温上升；10-13日云量增多，间中有零星小雨，气温逐日下降，早晚清凉。   深圳市气象台2025年11月07日10:00发布。\n\n## 深圳各区逐时预报\n\n###### 11月7日 10:05 更新\n\n罗湖\n福田\n南山\n宝安\n龙岗\n龙华\n光明\n坪山\n盐田\n大鹏\n深汕\n\n福田区\n\n# 23-28℃\n\n阴天间多云，偶有零星小雨\n\n 60-80 [...] | 预报日期 | 今天（11月6日） | 明天（11月7日） | 后天（11月8日） |\n ---  --- |\n| 空气污染气象条件等级预报 | 二级 较好较有利于空气污染物稀释、扩散和清除 | 三级 一般对空气污染物稀释、扩散和清除无明显影响 | 二级 较好较有利于空气污染物稀释、扩散和清除 |\n| 灰霾等级预报 | 

{'input': '请根据深圳未来三天的天气生成一份旅游攻略',
 'output': 'Agent stopped due to iteration limit or time limit.'}

#### AgentExecutor部分参数说明

- 使用ReAct模式时, 要求LLM的响应必须遵守严格的格式(如包含Thought、Action等标记), 但LLM会直接返回 非结构化的普通文本，导致解析器无法识别
    - 创建AgentExecutor时,添加 handle_parsing_errors=True,用于控制Agent在解析工具调用 或 输出时发生错误的容错能力
    - handle_parsing_errors=True
        - 自动捕获错误并修复: 当解析失败时,Agent不会直接崩溃,而是将错误信息传递给LLM,让LLM自行修正并重试
        - 降级处理: 如果重试后仍然失败,Agent会返回一个友好的错误消息(为False则会抛出异常)
        - 适用于生产环境,可保证鲁棒性

### 2）使用ChatPromptTemplate

- 如果要使用ChatPromptTemplate,则需要修改的提示词模板如下(其他不用改):

In [ ]:
prompt = ChatPromptTemplate.from_messages([
    ("system", """Answer the following questions as best you can. You have access to the
    following tools:
    {tools}
    Use the following format:
    Question: the input question you must answer
    Thought: you should always think about what to do
    Action: the action to take, should be one of [{tool_names}]
    Action Input: the input to the action
    Observation: the result of the action
    ... (this Thought/Action/Action Input/Observation can repeat N times)
    Thought: I now know the final answer
    Final Answer: the final answer to the original input question
    Begin!
    """),
    ("system", "当前思考：{agent_scratchpad}"),
    ("human", "我的问题是：{question}"), #必须在提示词模板中提供agent_scratchpad参数。
])

## 2.3、小结

- 下表总结了 Function Call 模式与 ReAct 模式的主要区别（结合上述代码示例）：

| 特性        | Function Call 模式                                | ReAct 模式                                |
|-----------|---------------------------------------------------|-------------------------------------------|
| 核心依赖      | LLM 原生函数调用能力（如 OpenAI Function Calling） | 提示词模板引导（无需原生函数调用支持）    |
| 输出格式      | 结构化指令（如 JSON）                              | 自然语言思考过程 + 结构化工具调用指令     |
| 灵活性       | 较低（适合单轮或简单多轮调用）                    | 较高（支持复杂多轮推理和工具调用）        |
| 可解释性      | 较低（仅展示工具调用，无思考过程）                 | 较高（显式输出思考过程）                  |
| 适配 LLM 范围 | 仅限支持函数调用的模型（如 GPT-3.5/4、Claude 3）  | 所有 LLM（包括开源模型如 Llama 2/3、Mistral）|
| 实现复杂度     | 低（无需复杂提示词，依赖 LLM 原生能力）            | 中（需设计 / 加载 ReAct 提示词模板）      |
| 提示词模板     | 推荐使用ChatPromptTemplate            | 推荐PromptTemplate       |
| 适用场景      | 简单工具调用、固定格式返回结果                     | 复杂推理、多轮工具调用、高可解释性需求     |

- 可见，Function Call 适用于模型本身原生支持函数调用的场景，语义结构化好，设置简便；而 ReAct 能力更灵活、更可解释，适用于复杂多轮推理和与各种开源模型的结合

# 3、Agent嵌入记忆组件

- 在创建AgentExecutor时, 传入记忆组件Memory

## 3.1、Function Call模式

- 如果使用的是FUNCTION_CALL方式，则创建Agent时，推荐使用ChatPromptTemplate

In [27]:
from langchain_classic.memory import ConversationBufferMemory, ConversationSummaryMemory, ConversationBufferWindowMemory

# 5. 定义提示词模板
prompt = ChatPromptTemplate.from_messages([
    ("system", "你是一个有用的助手，可以回答问题并使用工具。"),
    ("placeholder", "{chat_history}"), # 存储多轮对话的历史记录 如果你没有显式传入 chat_history，Agent 会默认将其视为空列表 []
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}")
])
# 6. 定义记忆组件(以ConversationBufferMemory为例)
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

# llm,tools,agent在前面的代码实践中都有定义(使代码看起来更简洁),需要运行前面有包含相关定义的代码
# 7.创建Agent对象
agent = create_tool_calling_agent(llm, tools, prompt)
# 8.创建AgentExecutor执行器对象(通过源码可知，memory参数声明在AgentExecutor父类中)
agent_executor = AgentExecutor(agent=agent,memory=memory ,tools=tools, verbose=True)

# 高质量测试Agent记忆能力的案例

# 场景1：给定个人信息，对Agent进行追问，测试其对多轮对话中用户“人物设定”的记忆
print("== 场景1：个人信息记忆测试 ==")
result1 = agent_executor.invoke({"input": "你好，我叫xulei，是清华大学的AI应用研究员，家有两只猫。"})
print(f"用户设定: {result1}")

result2 = agent_executor.invoke({"input": "你还记得我叫什么名字，职业是什么吗？"})
print(f"记忆追问: {result2}")

result3 = agent_executor.invoke({"input": "我的宠物是什么？"})
print(f"宠物追问: {result3}")

# 场景2：多步对话依赖，测试Agent对历史多轮输入的总结
print("\n== 场景2：多步对话总结能力测试 ==")
agent_executor.invoke({"input": "我早晨阅读写作完之后,就去健身房健身了"})
agent_executor.invoke({"input": "我上午在公司开会,下午要去图书馆还书，晚上陪家人吃饭。"})
result7 = agent_executor.invoke({"input": "请你总结一下我今天的主要安排。"})
print(f"生活安排总结: {result7}")

# 场景3：信息覆盖与更正，测试Agent对更新后记忆的准确性
print("\n== 场景3：记忆覆盖测试 ==")
agent_executor.invoke({"input": "我最喜欢的城市是武汉。"})
agent_executor.invoke({"input": "我后来更喜欢杭州了。"})
agent_executor.invoke({"input": "而现在我还是喜欢待在深圳了。"})
result8 = agent_executor.invoke({"input": "我最喜欢哪个城市？"})
print(f"覆盖后查询: {result8}")

== 场景1：个人信息记忆测试 ==


> Entering new AgentExecutor chain...
你好，xulei！很高兴认识你！作为清华大学的AI应用研究员，你一定在人工智能领域有很多有趣的研究和经验。两只猫一定给你的生活增添了不少乐趣吧！它们叫什么名字呢？或者有什么特别的故事可以分享吗？

> Finished chain.
用户设定: {'input': '你好，我叫xulei，是清华大学的AI应用研究员，家有两只猫。', 'chat_history': [HumanMessage(content='你好，我叫xulei，是清华大学的AI应用研究员，家有两只猫。', additional_kwargs={}, response_metadata={}), AIMessage(content='你好，xulei！很高兴认识你！作为清华大学的AI应用研究员，你一定在人工智能领域有很多有趣的研究和经验。两只猫一定给你的生活增添了不少乐趣吧！它们叫什么名字呢？或者有什么特别的故事可以分享吗？', additional_kwargs={}, response_metadata={})], 'output': '你好，xulei！很高兴认识你！作为清华大学的AI应用研究员，你一定在人工智能领域有很多有趣的研究和经验。两只猫一定给你的生活增添了不少乐趣吧！它们叫什么名字呢？或者有什么特别的故事可以分享吗？'}


> Entering new AgentExecutor chain...
当然记得！你叫 **xulei**，是 **清华大学的AI应用研究员**。如果有更多关于你的研究或生活想分享的，我很乐意听！

> Finished chain.
记忆追问: {'input': '你还记得我叫什么名字，职业是什么吗？', 'chat_history': [HumanMessage(content='你好，我叫xulei，是清华大学的AI应用研究员，家有两只猫。', additional_kwargs={}, response_metadata={}), AIMessage(content='你好，xulei！很高兴认识你！作为清华大学的AI应用研究员，你一定在人工智能领域有很多有趣的研究和经验。两只猫一定给你的生活增添了不少乐趣吧！它们叫什么名字呢？或

## 3.2、ReAct模式

- ReAct模式下，创建Agent时，可以使用ChatPromptTemplate、PromptTemplate

In [22]:
# 自定义提示词模版
template = '''助手是由 OpenAI 训练的一个大型语言模型。\n
   助手旨在能够协助完成各种各样的任务，从回答简单的问题到对各种主题进行深入的解释和讨论。作为语言模型，助手能够根据接收到的输入生成类似人类的文本，从而能够进行自然流畅的对话，并提供与当前主题相关且连贯的回答。\n
   助手在不断学习和改进，其能力也在不断发展。它能够处理和理解大量的文本，并利用这些知识为各种问题提供准确且有信息量的回答。此外，助手还能够根据接收到的输入生成自己的文本，从而能够参、与讨论，并对各种主题提供解释和描述。\n
   总的来说，助手是一个强大的工具，能够帮助完成各种各样的任务，并在各种主题上提供有价值的见解和信息。无论您是需要解答某个具体问题，还是只是想就某个特定话题进行交流，助手都在这里为您提供帮助。\n\n
工具：\n------
助手可以使用以下工具：\n
{tools}\n
若要使用某个工具，请使用以下格式：\n
```
思考：我是否需要使用工具？是\n
行动：要采取的行动，应为 [{tool_names}] 中的一个\n
行动输入：行动的输入\n
观察：行动的结果\n
```\n\n
当您有回复要对人类说，或者不需要使用工具时，您必须使用以下格式：\n
```\n
思考：我是否需要使用工具？否\n
最终答案：[您的回复在此]\n
```\n
开始！\n
之前的对话历史：\n
{chat_history}\n
新的输入：{input}\n
{agent_scratchpad}\n
\n
'''

prompt = PromptTemplate.from_template(template)

# 定义记忆组件(以ConversationBufferMemory为例)
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)
# 创建ReAct模式的Agent对象
agent = create_react_agent(llm, tools, prompt)
# 创建AgentExecutor执行器(推荐设置max_iterations,限制最大迭代次数，防止无限循环)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, memory=memory, handle_parsing_errors=True, max_iterations=1,max_execution_time=3)

# 测试
agent_executor.invoke({"input": "我最喜欢的城市是武汉。"})
agent_executor.invoke({"input": "我后来更喜欢杭州了。"})
agent_executor.invoke({"input": "而现在我还是喜欢待在深圳了。"})
result = agent_executor.invoke({"input": "我最喜欢哪个城市？"})
print(f"覆盖后查询: {result}")



> Entering new AgentExecutor chain...
思考：我是否需要使用工具？否

最终答案：武汉是一座充满活力和历史文化的城市，拥有丰富的美食、壮丽的景点以及深厚的文化底蕴。它是中国中部的重要城市，也是长江经济带的核心城市之一。无论是黄鹤楼、东湖风景区，还是热干面等特色美食，武汉都有许多值得喜爱的地方！很高兴知道您喜欢武汉！Invalid Format: Missing 'Action:' after 'Thought:'

> Finished chain.


> Entering new AgentExecutor chain...
思考：我是否需要使用工具？否

最终答案：杭州是一个非常美丽的城市，以其西湖、古典园林和丰富的文化底蕴而闻名。它的自然风光和现代化发展相结合，确实很吸引人。很高兴听到你喜欢杭州！Invalid Format: Missing 'Action:' after 'Thought:'

> Finished chain.


> Entering new AgentExecutor chain...
```
思考：我是否需要使用工具？否

最终答案：深圳是一个充满活力的城市，拥有快速发展的经济和丰富的文化氛围。无论是科技创新还是生活便利，深圳都吸引了许多人选择在那里生活。很高兴听到你喜欢深圳！如果你想聊聊深圳的某些方面，比如美食、景点或生活体验，我很乐意和你讨论。
```Invalid Format: Missing 'Action:' after 'Thought:'

> Finished chain.


> Entering new AgentExecutor chain...
思考：我是否需要使用工具？否

最终答案：根据之前的对话历史，您提到过喜欢武汉、杭州和深圳。虽然您提到过这些城市，但最终您表示还是喜欢待在深圳。因此，您目前最喜欢的城市应该是深圳。Invalid Format: Missing 'Action:' after 'Thought:'

> Finished chain.
覆盖后查询: {'input': '我最喜欢哪个城市？', 'chat_history': [HumanMessage(content='我最喜欢的城市是武汉。', additional_kwargs={}, 

## 3.3、远程获取提示词模板

- 以通用方式create_xxx_agent的ReAct模式为例，FUNCATION_CALL一样
- 远程的提示词模版可通过 https://smith.langchain.com/hub/hwchase17 获取
    - 举例：https://smith.langchain.com/hub/hwchase17/react-chat, 该模板是专为聊天场景设计的ReAct提示模板。这个模板中已经有聊天对话键 chat_history 、 agent_scratchpad

In [26]:
# 远程获取提示词模板
prompt = hub.pull("hwchase17/react-chat")

memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)
# 6.创建Agent、AgentExecutor
agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, memory=memory, verbose=True, max_iterations=3, max_execution_time=3)

result1 = agent_executor.invoke({"input": "深圳明天的天气怎么样？"})
print(f"result1: {result1}")

result2 = agent_executor.invoke({"input": "那上海呢？"})
print(f"result2: {result2}")



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes  
Action: tavily_search_results_json  
Action Input: 深圳明天的天气  
[{'title': '天气实况与预报-深圳市气象局（台）', 'url': 'https://weather.sz.gov.cn/mobile/qixiangfuwu/yubaofuwu/jinmingtianqiyubao/', 'content': '20时\n\nImage 61\n多云\n\n   21时\n\nImage 62\n多云\n\n   22时\n\nImage 63\n多云\n\n   23时\n\nImage 64: 多云间阴天，空气干燥\n阴\n\n   00时\n\nImage 65: 多云间阴天，空气干燥\n阴\n\n   01时\n\nImage 66: 多云间阴天，空气干燥\n阴\n\n   02时\n\nImage 67: 多云间阴天，空气干燥\n阴\n\n 深圳未来十天预报 \n\n11月10日 16:49 更新\n\n   明天\n\n11月11Image 68: 多云间阴天，空气干燥\n阴天间多云，有分散小雨\n\nImage 69\n北风\n\n4-5级55-85%\n   周三\n\n11月12Image 70: 阴天，有小雨\n阴天，有小雨\n\nImage 71\n北风\n\n4-5级60-90%\n   周四\n\n11月13Image 72: 多云间阴天，空气干燥\n多云\n\nImage 73\n北风\n\n3-4级55-85%\n   周五\n\n11月14Image 74: 多云，空气干燥\n多云间晴天\n\nImage 75\n东北风\n\n2-3级50-85%\n   周六 [...] 【深圳天气趋势】第26号台风“凤凰”10日14时在深圳东南方约754公里的南海海面上，预计11日夜间开始逐渐转向东北方向移动，强度逐渐减弱。受其和冷空气共同影响，预计11-12日阴天有小雨，东部沿海及高地最大阵风8-9级，气温逐日下降，天气较凉；展望13-17日多云到晴天，空气干燥。深圳市气象台2025年11月10日16:00发布。 \n\n